In [ ]:
macro bash_str(s) open(`bash`,"w",STDOUT) do io; print(io, s); end; end

In [ ]:
bash"""
cd ../3rd_Party_Code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/
julia Solve.jl
"""

In [ ]:
include("../3rd_Party_Code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/Include.jl")
include("../3rd_Party_Code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/Bounds.jl")
include("../3rd_Party_Code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/TXTLDictionary.jl")

# load the data dictionary -
data_dictionary = DataDictionary(0,0,0)
TXTL_dictionary = TXTLDictionary(0,0,0,data_dictionary)

#Set objective reaction
data_dictionary["objective_coefficient_array"][find_idx("but_export", data_dictionary)] = -1;

#=============================Cases=========================================#
#Define case number
# 1 = Amino Acid Uptake & Synthesis
# 2 = Amino Acid Uptake w/o Synthesis
# 3 = Amino Acid Synthesis w/o Uptake
Case = 1
if Case == 1
  data_dictionary["AASyn"] = 100;
  data_dictionary["AAUptake"] = 30
  data_dictionary["AASecretion"] = 0;
end
if Case == 2
  data_dictionary["AASyn"] = 0;
  data_dictionary["AAUptake"] = 30
  data_dictionary["AASecretion"] = 0;
end
if Case == 3
  data_dictionary["AASyn"] = 100;
  data_dictionary["AAUptake"] = 0
  data_dictionary["AASecretion"] = 100;
end

#Set Promoter
#1 = T7 Promoter model
#2 = P70a Promoter model
Promoter_model = 1
#===========================================================================#

#Set Plasmid Dose (nM)
plasmid_concentration = 13.3;
volume = TXTL_dictionary["volume"]
gene_copy_number = (volume/1e9)*(6.02e23)*plasmid_concentration;
TXTL_dictionary["gene_copies"] = gene_copy_number

#Set Glucose and Oxygen (mM/h)
data_dictionary["GlcUptake"] = 30
data_dictionary["Oxygen"] = 100;

# solve the lp problem -
data_dictionary = Bounds(data_dictionary,TXTL_dictionary);
(objective_value, flux_array, dual_array, uptake_array, exit_flag) = FluxDriver(data_dictionary)
include("../3rd_Party_Code/Sequence-Specific-FBA-CFPS-Publication-Code/Model/Performance.jl")

In [ ]:
data_dictionary

In [ ]:
data_dictionary["list_of_reaction_strings"]

In [ ]:
outfile = open("../BTOL/Bounds.txt", "w")
writedlm(outfile,data_dictionary["default_flux_bounds_array"]);
close(outfile);